In [ ]:
%cd ../src

import os
from common.utils import Params, set_logger, copy_weight, load_checkpoint, save_checkpoint_pos_ori, write_log, get_lr, write_train_summary_scalars, write_val_summary_joint, change_momentum
from features.networks import TGraphNet, TGraphNetSeq
from vizualization.vizualize import plot_adjacency_matrix, plot_pose_animation, plot_poses_only
from common.h36m_skeleton import get_node_names, get_edge_names
import torch
from graph import Graph
import numpy as np

from data.h36m_dataset import Human36M
from data.pw3d_dataset import PW3D
from common.h36m_skeleton import joint_id_to_names, joint_names
from data.generators import ChunkedGenerator_Seq, UnchunkedGenerator_Seq, ChunkedGenerator_Frame, ChunkedGenerator_Seq2Seq, eval_data_prepare
from evaluation import *
from einops import rearrange

import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import json

# Experiment details

In [ ]:
exp = "run13"
description = "baseline_root_rel"
exp_desc = "eval root rel model" 
#exp_desc= "eval cvpr_v2/run4_1/adjacency matrices"

# Load parameters
json_path = os.path.join('../models/stgcn/root_rel/params.json')
assert os.path.isfile(json_path), "No json file found at {}".format(json_path)
params = Params(json_path)

# CUDA settings
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#params.batch_size = 1
data_dir = "/media/HDD3/datasets/Human3.6M/pose_zip"

# Load model

In [ ]:
model = TGraphNetSeq(infeat_v=params.input_node_feat,
                  infeat_e=params.input_edge_feat,
                  nhid_v=params.num_hidden_nodes,
                  nhid_e=params.num_hidden_edges,
                  n_oute=params.output_edge_feat,
                  n_outv=params.output_node_feat,
                  gcn_window=params.gcn_window,
                  tcn_window=params.tcn_window,
                  in_frames=params.in_frames,
                  gconv_stages=params.gconv_stages,
                  num_groups=params.num_groups,
                  dropout=params.dropout,
                  aggregate=params.aggregate,
                  use_residual_connections=params.use_residual_connections,
                  use_non_parametric=params.use_non_parametric,
                  use_edge_conv=params.use_edge_conv,
                  learn_adj=False)

load_checkpoint('../models/stgcn/root_rel/best_pos.pth.tar', model)

model.eval()


In [ ]:
actions = ['Directions', 'Discussion','Eating', 'Greeting','Phoning','Photo', 'Posing', 'Purchases','Sitting', 'SittingDown', 'Smoking', 'Waiting','WalkDog','WalkTogether','Walking', 'all']
# actions = ['all']

# Evaluating metrics
metrics = [ mpjpe, p_mpjpe, mean_velocity_error ]


# Compute error

In [ ]:
model.cuda().eval()

In [ ]:
def per_axis_mpjpe(pred, gt):
    err_x = torch.norm(pred[:, :, :, 0] - gt[:, :, :, 0], dim = -1)
    err_y = torch.norm(pred[:, :, :, 1] - gt[:, :, :, 1], dim = -1)
    err_z = torch.norm(pred[:, :, :, 2] - gt[:, :, :, 2], dim = -1)

    return torch.mean(err_x), torch.mean(err_y), torch.mean(err_z)

In [ ]:
per_axis_mpjpe(torch.zeros(32, 81, 1, 3), torch.zeros(32, 81, 1, 3))

In [ ]:

# Log dict
joint_errors = {}
action_errors = {}
action_errors_traj = {}

for a in actions:
    action_errors[a] = np.empty([0])
    action_errors_traj[a] = np.empty([0])

log_dict = {}
log_dict['exp_name'] = exp  
log_dict['exp_desc'] = exp_desc
log_dict['restore_weight'] = json_path.__str__()

# Loop over action
for act in actions:
    print(act)
    test_dataset = Human36M(data_dir="/media/HDD3/datasets/Human3.6M/pose_zip", train=False, ds_category=params.ds_category, actions=act)

    cam, pos2d, pos3d, angles_6d, edge_features = test_dataset.cam, test_dataset.pos2d, test_dataset.pos3d_centered, [], []
    val_generator = ChunkedGenerator_Seq2Seq(params.batch_size, cameras=None, poses_2d=pos2d, poses_3d=pos3d,
                                                   chunk_length=31, pad=25, out_all=True, shuffle=False,
                                                   augment=False, reverse_aug=False,)

    # joint id to name mapping
    joint_id_to_names = test_dataset.joint_id_to_names
    total_errors = np.empty([0])
    total_errors_traj = np.empty([0])
    for j in joint_names:
        joint_errors[j] = np.empty([0])

    # model forward
    err_val_p2_list = []
    summary = []
    joint_dict = joint_id_to_names

    cnt = 0

    total_mpjpe = 0
    total_pmpjpe = 0
    total_err_traj = 0
    total_velocity = 0
    total_err_x = 0
    total_err_y = 0
    total_err_z = 0
    N = 0
    n_batches = 0
    per_joint_err = torch.zeros(17)

    with torch.no_grad():
        for cameras_val, batch_3d, batch_6d, batch_2d, batch_edge in val_generator.next_epoch():
            input_2d = torch.FloatTensor(batch_2d).to(device)
            target_pose_3d = torch.FloatTensor(batch_3d).to(device)
            # cameras_val = torch.from_numpy(cameras_val.astype('float32')).to(device)

            middle_index = int((target_pose_3d.shape[1] - 1) / 2)
            pad = 15
            start_index = middle_index - pad
            end_index = middle_index + pad + 1
            B, T, J, D = target_pose_3d.shape

            predicted_pos3d = model(input_2d)
            predicted_pos3d_center = predicted_pos3d[:, start_index:end_index].reshape(B, (2 * pad + 1), J, D)

            # target_angle_6d = target_angle_6d[:, middle_index].view_as(predicted_angle_6d)
            target_pose_3d = target_pose_3d.view_as(predicted_pos3d)
            target_pose_3d_center = target_pose_3d[:, start_index:end_index].reshape(B, (2 * pad + 1), J, D)

            target_pose_3d_center[:, :, :1] = 0
            predicted_pos3d_center[:, :, :1] = 0

            errors = torch.norm(target_pose_3d_center - predicted_pos3d_center, dim=len(predicted_pos3d_center.shape)-1)
            err_x, err_y, err_z = per_axis_mpjpe(target_pose_3d_center[:, :, :1], predicted_pos3d_center[:, :, :1])

            total_err_x += err_x * predicted_pos3d.shape[0] * predicted_pos3d.shape[1]
            total_err_y += err_y * predicted_pos3d.shape[0] * predicted_pos3d.shape[1]
            total_err_z += err_z * predicted_pos3d.shape[0] * predicted_pos3d.shape[1]
            # errors: [B, T, N]
            total_errors = np.concatenate((total_errors, errors.mean(dim=-1).reshape(-1).cpu().data.numpy()))
            total_errors_traj = np.concatenate((total_errors_traj, errors[:, :, :1].mean(dim=-1).reshape(-1).cpu().data.numpy()))
            action_errors[act] = np.concatenate((action_errors[act], errors.mean(dim=-1).reshape(-1).cpu().data.numpy()))
            action_errors_traj[act] = np.concatenate((action_errors_traj[act], errors[:, :, :1].mean(dim=-1).reshape(-1).cpu().data.numpy()))
            errors = rearrange(errors, 'B T N -> N (B T)').cpu().data.numpy()
            for joint_id in range(errors.shape[0]):
                joint_errors[joint_dict[joint_id]] = np.concatenate((joint_errors[joint_dict[joint_id]], errors[joint_id]), axis=0)

            n_batches += 1

            err_pos, joint_err = metrics[0](
                predicted_pos3d_center.cpu().data,
                target_pose_3d_center.cpu().data
            )

            err_vel = metrics[2](
                predicted_pos3d_center.cpu().data,
                target_pose_3d_center.cpu().data,
                1
            )

            total_mpjpe += err_pos.cpu().data.numpy() * predicted_pos3d.shape[0] * predicted_pos3d.shape[1]
            per_joint_err += joint_err * predicted_pos3d.shape[0] * predicted_pos3d.shape[1]
            N += predicted_pos3d.shape[0] * predicted_pos3d.shape[1]

            total_velocity += err_vel.cpu().data.numpy() * predicted_pos3d.shape[0] * predicted_pos3d.shape[1]
            # p-mpjpe
            err_pos_p = metrics[1](
                predicted_pos3d_center.cpu().data.numpy().reshape(-1, 17, 3),
                target_pose_3d_center.cpu().data.numpy().reshape(-1, 17, 3)
            )

            total_pmpjpe += err_pos_p * predicted_pos3d.shape[0] * predicted_pos3d.shape[1]

            err_traj, _ = metrics[0](
                target_pose_3d_center[:, :, :1].cpu().data,
                predicted_pos3d_center[:, :, :1].cpu().data
            )

            total_err_traj += err_traj.cpu().data.numpy() * predicted_pos3d.shape[0] * predicted_pos3d.shape[1]


    # Average scores 

    # joint level err
    # log entry
    log_dict[act] = {}
    log_dict[act]['p1_err'] = total_mpjpe / N
    log_dict[act]['p2_err'] = total_pmpjpe / N
    log_dict[act]['traj_err'] = total_err_traj / N
    log_dict[act]['velocity_err'] = total_velocity / N
    log_dict[act]['err_x'] = total_err_x / N
    log_dict[act]['err_y'] = total_err_y / N
    log_dict[act]['err_z'] = total_err_z / N

    mean_err_joint = per_joint_err / N

    log_dict[act]['p1_err_joint']={}
    for joint_id in range(len(mean_err_joint)):
        log_dict[act]['p1_err_joint'][joint_dict[joint_id]] = {}
        log_dict[act]['p1_err_joint'][joint_dict[joint_id]]['mean'] = mean_err_joint[joint_id].cpu().data.numpy().item()


In [ ]:
log_dict

In [ ]:
action_errors_traj

In [ ]:
import json
# write log to file
log_dict_fname = os.path.join("../reports/evaluation_results/", f"eval_action_{exp}_{description}_root_included.json")
with open(log_dict_fname, 'w') as fp:
        json.dump(log_dict, fp, sort_keys=True, indent=4) 
        

In [ ]:
import json
# read log file from disk
log_dict_fname = os.path.join("../reports/evaluation_results/", f"eval_action_run14_trajectory_model.json")
with open(log_dict_fname, 'r') as fp:
    log_dict = json.load(fp)


# Plot Joint Wise Bar Plot
## MPJPE Error - Latex table

In [ ]:
def defaultPlotting(): sns.set(rc={'figure.figsize':(11.7,8.27),"font.size":20,"axes.titlesize":20,"axes.labelsize":20},style="whitegrid")

In [ ]:
errors_dup = total_errors.copy()

In [ ]:
total_errors

In [ ]:
# errors_dup[errors_dup > 250] = 250
errors_dup[errors_dup < 25] = 20
errors_dup[errors_dup > 75] = 71

In [ ]:
# plt.figure(figsize=(12, 8))
defaultPlotting()

ax = sns.histplot(x=errors_dup, fill="blue", element="step", stat="percent", binwidth=5, binrange=[20, 75])
# ax = sns.violinplot(x=errors_dup,)
ax.set_xticks([20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75])
# ax.set_xticklabels([0, 0, 50, 100, 150, 200, "> 250"], fontsize=20)
ax.set_xlabel("The Range of MPJPE (mm)", fontsize=24, fontdict={'weight': 'bold'})
ax.set_title("Distribution of MPJPE", fontsize=24, fontdict={'weight': 'bold'})
ax.set_ylabel("Percent", fontsize=24, fontdict={'weight': 'bold'})
ax.set_yticklabels(ax.get_yticks(), fontsize=20)
ax.set_xticklabels(["< 25", 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, "> 75"], fontsize=20)

plt.savefig("../reports/figures/mpjpe_dist.pdf")
plt.show()

In [ ]:
font = {'weight': 'bold'}
defaultPlotting()

for a in actions:    
    errors_dup = action_errors[a].copy()
    # errors_dup[errors_dup > 250] = 250
    errors_dup[errors_dup < 25] = 20
    errors_dup[errors_dup > 75] = 71

    ax = sns.histplot(x=errors_dup, fill="blue", bins=[20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75], element="poly", stat="percent",)
    # ax = sns.violinplot(x=errors_dup)
    # ax.set_xlim([0, 250])
    ax.set_xticks([20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75])
    ax.set_xticklabels(["< 25", 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, "> 75"], fontsize=20)
    ax.set_xlabel("MPJPE (mm)", fontsize=24, fontdict=font)
    ax.set_title(f"Distribution of MPJPE ({a})", fontsize=24, fontdict=font)
    ax.set_ylabel("Percent", fontsize=24, fontdict=font)
    ax.set_yticklabels(ax.get_yticks(), fontsize=20)

    plt.savefig(f"../reports/figures/mpjpe_dist_{a}_hplot.png", dpi=300, bbox_inches='tight', pad_inches = 0)
    plt.show()

In [ ]:
# actions = ['WalkTogether']
actions = ['Directions', 'Discussion','Eating', 'Greeting','Phoning','Photo', 'Posing', 'Purchases',\
   'Sitting', 'SittingDown', 'Smoking', 'Waiting','WalkDog','Walking','WalkTogether', 'all']

# print on screen
print("action", "p1", "p2")
for act in actions:
    act_scores = log_dict[act]
    #print(act_scores)
    print(act, ",", np.round(act_scores['p1_err'], 2), ",", np.round(act_scores['p2_err'], 2))

# for latex
header = "\\textbf{Protocol \#1} \t"
score_rec_p1 = "Ours (P1) \t"
score_rec_p2 = "Ours (P2) \t"
score_rec_vel = "Ours (Vel) \t"

for act in actions:
    act_scores = log_dict[act]
    header+=" & "+act
    score_rec_p1+=" & "+str(np.round(act_scores['p1_err'], 1))
    score_rec_p2+=" & "+str(np.round(act_scores['p2_err'], 1))
    score_rec_vel+=" & "+str(np.round(act_scores['velocity_err'], 1))


print(header)
print(score_rec_p1)
print(score_rec_p2)
print(score_rec_vel)


In [ ]:
# actions = ['WalkTogether']
actions = ['Directions', 'Discussion','Eating', 'Greeting','Phoning','Photo', 'Posing', 'Purchases',\
   'Sitting', 'SittingDown', 'Smoking', 'Waiting','WalkDog','Walking','WalkTogether']

# print on screen
print("action", "p1", "p2")
for act in actions:
    act_scores = log_dict[act]
    #print(act_scores)
    print(act, ",", np.round(act_scores['p1_err'], 2), ",", np.round(act_scores['p2_err'], 2))

# for latex
header = "\\textbf{Protocol \#1} \t"
score_rec_p1 = "Ours (P1) \t"
score_rec_p2 = "Ours (P2) \t"
score_rec_vel = "Ours (Vel) \t"

for act in actions:
    act_scores = log_dict[act]
    header+=" & "+act
    score_rec_p1+=" & "+str(np.round(act_scores['p1_err'], 1))
    score_rec_p2+=" & "+str(np.round(act_scores['p2_err'], 1))
    score_rec_vel+=" & "+str(np.round(act_scores['velocity_err'], 1))


print(header)
print(score_rec_p1)
print(score_rec_p2)
print(score_rec_vel)


In [ ]:
res_dict = {}

for jname, jid in zip(joint_names, joint_id_to_names.keys()):
    errs = joint_errors[jname]
    mu, std = np.mean(errs), np.std(errs)
    res_dict[jname] = [mu, std / 10]

In [ ]:
res_dict = {}

for act in actions:
    if act == ""
    errs = action_errors_traj[act]
    mu, std = np.mean(errs), np.std(errs)
    res_dict[act] = [mu, std / 10]

In [ ]:
# sns.set(font_scale = 1.4)
sns.set_style("whitegrid")
plt.figure(figsize=(12, 9))
# defaultPlotting()
ax = sns.barplot(x=list(res_dict.keys())[1:], y=list(map(lambda v: v[0], res_dict.values()))[1:], color='steelblue')

font = {'weight': 'bold'}
ax.axes.set_title("Per Action Errors",fontsize=20, fontdict={'weight': 'bold'})
ax.set_xlabel("Action",fontsize=20, fontdict=font)
ax.set_ylabel("MPJPE",fontsize=20, fontdict=font)
ax.tick_params(labelsize=20)
ax.tick_params(axis='x', rotation=90)

plt.savefig("../reports/figures/per_action_err_traj.pdf",bbox_inches='tight')
plt.show()

In [ ]:
# sns.set(font_scale = 1.4)
sns.set_style("whitegrid")
plt.figure(figsize=(12, 9))
# defaultPlotting()
ax = sns.barplot(x=list(res_dict.keys())[1:], y=list(map(lambda v: v[0], res_dict.values()))[1:], color='steelblue')

font = {'weight': 'bold'}
ax.axes.set_title("Per Joint Errors",fontsize=24, fontdict={'weight': 'bold'})
ax.set_xlabel("Action",fontsize=24, fontdict=font)
ax.set_ylabel("MPJPE",fontsize=24, fontdict=font)
ax.tick_params(labelsize=22)
ax.tick_params(axis='x', rotation=60)

plt.savefig("../reports/figures/per_joint_err.pdf",bbox_inches='tight')
plt.show()

# Inference Time:

In [ ]:

dummy_input1 = torch.randn(1, 81, 17, 2, dtype=torch.float).to(device)

# INIT LOGGERS
starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
repetitions = 1000
timings=np.zeros((repetitions,1))
#GPU-WARM-UP
for _ in range(10):
    _ = model(dummy_input1)

# MEASURE PERFORMANCE
with torch.no_grad():
    for rep in range(repetitions):
        starter.record()
        _ = model(dummy_input1)
        ender.record()
        # WAIT FOR GPU SYNC
        torch.cuda.synchronize()
        curr_time = starter.elapsed_time(ender)
        timings[rep] = curr_time
mean_syn = np.sum(timings) / repetitions
std_syn = np.std(timings)
print(mean_syn, " ms") 

# FLOPS

In [ ]:
from fvcore.nn import FlopCountAnalysis

In [ ]:
flops = FlopCountAnalysis(model.cpu(), torch.randn(1, 81, 17, 2, dtype=torch.float).cpu())
flops.total() / 81

In [ ]:
flops.by_operator(), flops.by_module()